In [1]:
import math

def init_computer(code, inputs):
    return {
        'mem': code.copy(),
        'mem_size': len(code),
        'extend_mem' : {},
        'inst': 0,
        'rel': 0,
        'inputs': inputs.copy(),
        'outputs': [],
        'halt': False,
        'needs_input': False
    }

def read_mem(computer, pos):
    if(pos >= computer['mem_size']):
        if(pos in computer['extend_mem']):
            return computer['extend_mem'][pos]
        else:
            return 0
    else:
        return computer['mem'][pos]

def write_mem(computer, pos, val):
    if(pos < 0):
        print("invalid mem pos %i" % pos)
        return
    if(pos >= computer['mem_size']):
        computer['extend_mem'][pos] = val
    else:
        computer['mem'][pos] = val
    #if(pos == 386):
    #    print("writing to mem[386] = % i, rel[%i]+1 = %i, extend=" % (val, computer['rel'], read_mem(computer, computer['rel']+1)) , end = "")
    #    print(computer['extend_mem'])
        

def run(computer):
    code_size = len(computer['mem'])
    i = computer['inst']
    outputs = []
    op_info = {1:3, 2:3, 3:1, 4:1, 5:2, 6:2, 7:3, 8:3, 9:1, 99:0}
    computer['needs_input'] = False
    while(True):
        op = read_mem(computer, i)
        opcode = op % 100
        if(not(opcode in op_info)):
            print("error unknown opcode %i" % (opcode))
            computer['needs_input'] = False
            break
        a0 = -1
        a1 = -1
        a2 = -1
        jump = False
        if(op_info[opcode] > 0):
            p_mode = (math.floor(op / 100) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a0 = read_mem(computer, i + 1)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a0 = i + 1
            elif( p_mode == 2 ):
                #relative mode
                a0 = read_mem(computer, i + 1) + computer['rel']
        if(op_info[opcode] > 1):
            p_mode = (math.floor(op / 1000) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a1 = read_mem(computer, i + 2)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a1 = i + 2
            elif( p_mode == 2 ):
                #relative mode
                a1 = read_mem(computer, i + 2) + computer['rel']
        if(op_info[opcode] > 2):
            p_mode = (math.floor(op / 10000) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a2 = read_mem(computer, i + 3)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a2 = i + 3
            elif( p_mode == 2 ):
                #relative mode
                a2 = read_mem(computer, i + 3) + computer['rel']
        if(opcode == 1):
            #add op
            write_mem(computer, a2, read_mem(computer, a0) + read_mem(computer, a1))
        elif(opcode == 2):
            #mult op
            write_mem(computer, a2, read_mem(computer, a0) * read_mem(computer, a1))
        elif(opcode == 3):
            #read op
            if(len(computer['inputs']) == 0):
                computer['needs_input'] = True
                break
            write_mem(computer, a0, computer['inputs'][0])
            computer['inputs'] = computer['inputs'][1:]
        elif(opcode == 4):
            #write op
            #if(read_mem(computer, a0) == -1):
                #print("outputting -1 at inst: %i" %i)
            outputs.append(read_mem(computer, a0))
        elif(opcode == 5):
            #jump if true op
            if(read_mem(computer, a0) != 0):
                jump = True
                i = read_mem(computer, a1)
        elif(opcode == 6):
            #jump if false op
            if(read_mem(computer, a0) == 0):
                jump = True
                i = read_mem(computer, a1)
        elif(opcode == 7):
            #check less than op
            write_mem(computer, a2, 1 if(read_mem(computer, a0) < read_mem(computer, a1)) else 0)
        elif(opcode == 8):
            #check equals op
            write_mem(computer, a2, 1 if(read_mem(computer, a0) == read_mem(computer, a1)) else 0)
        elif(opcode == 9):
            #change relative param op
            computer['rel'] = computer['rel'] + read_mem(computer, a0)
        elif(opcode == 99):
            #halt op
            computer['halt'] = True
            computer['needs_input'] = False
            break
        if(not(jump)):
            i = i + op_info[opcode] + 1
        if(i >= code_size):
            print('exiting b/c end of code reached')
            computer['needs_input'] = False
    computer['outputs'] = outputs
    computer['inst'] = i
    
    return computer


In [2]:
class vec2i(object):
    def __init__(self, x, y):
        self.x = int(round(x))
        self.y = int(round(y))
    
    #@classmethod
    #def from_vec2f(cls, vf):
        #return cls(int(round(vf.x)), int(round(vf.y)))

    def index(self):
        xpart = ""
        if(self.x < 0):
            xpart = "N%i" % abs(self.x) 
        else:
            xpart = "P%i" % abs(self.x)
        ypart = ""
        if(self.y < 0):
            ypart = "N%i" % abs(self.y) 
        else:
            ypart = "P%i" % abs(self.y)
        return "%s%s" % (xpart,ypart)
    
    def step(self, v2):
        return vec2i(self.x + v2.x, self.y + v2.y)
    
    def copy(self):
        return vec2i(self.x, self.y)
    
    def __str__(self):
        return ("(%i,%i)" % (self.x, self.y))
    
    def __repr__(self):
        return ("(%i,%i)" % (self.x, self.y))

class Screen(object):
    def __init__(self):
        self.pixels = {}
        self.leftmost = 0
        self.upmost = 0
        self.rightmost = 0
        self.downmost = 0
        
    def tile(self, v):
        key = v.index()
        if(not(key in self.pixels)):
            return 0
        return self.pixels[key]
    
    def set_tile(self, v, tile):
        key = v.index()
        self.pixels[key] = tile
        if(v.x < self.leftmost):
            self.leftmost = v.x
        if(v.y < self.upmost):
            self.upmost = v.y
        if(v.x > self.rightmost):
            self.rightmost = v.x
        if(v.y > self.downmost):
            self.downmost = v.y
        
    def count_tiled(self):
        return len(self.pixels.keys())
    
    def render(self):
        width = self.rightmost - self.leftmost + 1
        height = self.downmost - self.upmost + 1
        image = ''
        for y in range(height):
            for x in range(width):
                cursor = vec2i(x + self.leftmost, y + self.upmost)
                tileind = self.tile(cursor)
                if(tileind == 0):
                    image = image + ' '
                elif(tileind == 1): #wall
                    image = image + '%'
                elif(tileind == 2): #block
                    image = image + '#'
                elif(tileind == 3): #paddle
                    image = image + '='
                elif(tileind == 4): #ball
                    image = image + '*'
            image = image + '\n'
        return image
    
    def count(self, tile):
        width = self.rightmost - self.leftmost + 1
        height = self.downmost - self.upmost + 1
        count = 0
        for y in range(height):
            for x in range(width):
                cursor = vec2i(x + self.leftmost, y + self.upmost)
                if(self.tile(cursor) == tile):
                    count = count + 1
        return count
                
                
        
class Game(object):
    def __init__(self, code, screen):
        self.computer = init_computer(code, [])
        self.score = 0
        self.screen = screen
        self.posx = 21
        
    def exec1(self):
        self.computer =  run(self.computer)
        outputs = self.computer['outputs']
        self.computer['outputs'] = []
        for i in range(0, len(outputs), 3):
            o0 = outputs[i]
            o1 = outputs[i+1]
            o2 = outputs[i+2]
            if(o0 == -1 and o1 == 0):
                self.score = o2
            else:
                self.screen.set_tile( vec2i(o0, o1), o2 )
    
    def execframe(self, show = True):
        while(not(self.computer['halt'] or self.computer['needs_input'])):
            self.exec1()
        if(show):
            print(self.screen.render())
            print("score: %i" % self.score)
            #print("mem[386]=%i, mem[387]=%i, rel[%i]+1 = %i" % (read_mem(self.computer,386), 
            #                                                    read_mem(self.computer,387),
            #                                                    self.computer['rel'],
            #                                                    read_mem(self.computer,self.computer['rel']+1)) )
        if(self.computer['halt']):
            print("halted!")
    
    def joyl(self, show = True):
        self.computer['needs_input'] = False
        self.computer['inputs'].append(-1)
        self.execframe(show)
        self.posx = self.posx - 1

    def joyr(self, show = True):
        self.computer['needs_input'] = False
        self.computer['inputs'].append(1)
        self.execframe(show)
        self.posx = self.posx + 1
    
    def joyn(self, show = True):
        self.computer['needs_input'] = False
        self.computer['inputs'].append(0)
        self.execframe(show)
        
    def play_self(self, show = True):
        ballx = read_mem(self.computer, 2533)
        if(ballx > self.posx):
            #print('moving right')
            self.joyr(show)
        elif(ballx < self.posx):
            #print('moving left')
            self.joyl(show)
        else:
            self.joyn(show)

    def exec(self):
        while(not(self.computer['halt'])):
            self.exec1()



In [3]:
screen = Screen()
code = [1,380,379,385,1008,2531,381039,381,1005,381,12,99,109,2532,1101,0,0,383,1101,0,0,382,21002,382,1,1,21002,383,1,2,21101,0,37,0,1106,0,578,4,382,4,383,204,1,1001,382,1,382,1007,382,43,381,1005,381,22,1001,383,1,383,1007,383,22,381,1005,381,18,1006,385,69,99,104,-1,104,0,4,386,3,384,1007,384,0,381,1005,381,94,107,0,384,381,1005,381,108,1106,0,161,107,1,392,381,1006,381,161,1101,-1,0,384,1105,1,119,1007,392,41,381,1006,381,161,1102,1,1,384,20101,0,392,1,21102,20,1,2,21101,0,0,3,21101,138,0,0,1106,0,549,1,392,384,392,20102,1,392,1,21102,20,1,2,21102,1,3,3,21101,0,161,0,1105,1,549,1101,0,0,384,20001,388,390,1,21001,389,0,2,21102,180,1,0,1105,1,578,1206,1,213,1208,1,2,381,1006,381,205,20001,388,390,1,20101,0,389,2,21101,205,0,0,1106,0,393,1002,390,-1,390,1102,1,1,384,20101,0,388,1,20001,389,391,2,21101,0,228,0,1106,0,578,1206,1,261,1208,1,2,381,1006,381,253,21001,388,0,1,20001,389,391,2,21102,253,1,0,1106,0,393,1002,391,-1,391,1101,0,1,384,1005,384,161,20001,388,390,1,20001,389,391,2,21101,0,279,0,1105,1,578,1206,1,316,1208,1,2,381,1006,381,304,20001,388,390,1,20001,389,391,2,21101,304,0,0,1106,0,393,1002,390,-1,390,1002,391,-1,391,1101,0,1,384,1005,384,161,21002,388,1,1,21002,389,1,2,21101,0,0,3,21102,1,338,0,1106,0,549,1,388,390,388,1,389,391,389,21002,388,1,1,20101,0,389,2,21101,4,0,3,21102,1,365,0,1106,0,549,1007,389,21,381,1005,381,75,104,-1,104,0,104,0,99,0,1,0,0,0,0,0,0,291,19,17,1,1,21,109,3,22101,0,-2,1,21202,-1,1,2,21102,0,1,3,21102,1,414,0,1105,1,549,22102,1,-2,1,22101,0,-1,2,21102,1,429,0,1106,0,601,1202,1,1,435,1,386,0,386,104,-1,104,0,4,386,1001,387,-1,387,1005,387,451,99,109,-3,2106,0,0,109,8,22202,-7,-6,-3,22201,-3,-5,-3,21202,-4,64,-2,2207,-3,-2,381,1005,381,492,21202,-2,-1,-1,22201,-3,-1,-3,2207,-3,-2,381,1006,381,481,21202,-4,8,-2,2207,-3,-2,381,1005,381,518,21202,-2,-1,-1,22201,-3,-1,-3,2207,-3,-2,381,1006,381,507,2207,-3,-4,381,1005,381,540,21202,-4,-1,-1,22201,-3,-1,-3,2207,-3,-4,381,1006,381,529,22101,0,-3,-7,109,-8,2106,0,0,109,4,1202,-2,43,566,201,-3,566,566,101,639,566,566,1202,-1,1,0,204,-3,204,-2,204,-1,109,-4,2106,0,0,109,3,1202,-1,43,593,201,-2,593,593,101,639,593,593,21002,0,1,-2,109,-3,2106,0,0,109,3,22102,22,-2,1,22201,1,-1,1,21102,479,1,2,21101,0,201,3,21101,946,0,4,21102,630,1,0,1105,1,456,21201,1,1585,-2,109,-3,2106,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,2,2,0,2,0,2,2,2,0,2,0,2,2,2,2,0,0,0,2,2,0,2,2,0,0,2,0,2,2,0,2,2,2,2,2,0,1,1,0,2,0,2,2,2,2,2,2,0,2,2,0,2,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,2,0,2,0,2,0,0,0,0,0,0,1,1,0,2,2,0,2,0,0,0,2,2,0,2,0,0,0,2,0,0,0,0,2,2,2,0,2,0,0,0,2,0,2,2,0,0,2,0,2,0,2,2,0,1,1,0,0,2,0,2,0,0,2,2,2,2,2,0,2,0,2,2,0,0,0,2,2,0,0,2,2,0,2,2,2,0,0,2,0,2,0,2,0,0,2,0,1,1,0,2,0,2,0,2,2,2,0,0,0,0,0,0,2,0,2,0,0,2,0,2,0,0,2,0,0,0,0,2,2,0,2,2,0,0,2,0,0,0,0,1,1,0,0,0,2,2,2,2,0,2,2,2,2,0,2,0,0,2,2,0,2,2,2,0,0,0,2,2,0,0,2,2,0,0,0,0,2,2,0,0,2,0,1,1,0,0,0,0,0,0,2,2,2,2,2,2,0,0,2,2,0,0,2,2,2,0,0,2,2,2,0,2,0,2,0,0,2,0,2,2,2,0,2,0,0,1,1,0,2,0,0,0,0,2,2,2,0,0,2,0,0,2,0,2,2,2,2,0,2,0,2,0,0,0,0,2,0,2,2,0,2,2,0,2,2,0,2,0,1,1,0,0,2,2,0,2,2,0,2,0,2,0,2,0,2,0,0,0,0,2,2,2,2,0,2,0,2,2,0,2,0,2,2,0,2,2,0,2,2,0,0,1,1,0,0,2,0,0,2,2,2,0,2,0,0,2,2,0,2,2,0,0,0,2,2,0,2,2,2,0,0,2,0,0,0,0,2,2,2,0,0,0,2,0,1,1,0,0,2,2,0,0,2,0,2,2,2,2,0,2,2,2,2,2,2,2,2,2,0,2,2,0,2,2,0,2,0,2,2,0,2,2,0,2,2,2,0,1,1,0,0,0,2,2,2,0,2,0,0,0,0,2,2,2,2,2,2,2,2,0,0,0,2,2,2,2,2,0,0,2,0,0,0,0,2,2,0,2,2,0,1,1,0,2,0,0,2,2,0,2,0,0,0,2,2,2,2,2,2,0,2,2,0,2,0,2,2,0,2,2,0,2,2,2,2,2,0,0,2,0,0,0,0,1,1,0,2,0,0,2,0,2,0,0,2,0,0,2,2,2,2,2,0,2,0,2,2,0,2,2,0,0,2,2,0,0,0,2,2,0,2,2,0,2,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,37,59,35,82,55,63,50,72,81,61,59,5,1,69,3,36,79,19,94,73,56,24,20,10,1,25,20,49,14,41,74,10,1,48,97,35,54,11,81,35,36,54,58,49,82,25,96,37,51,26,65,35,51,78,95,58,66,62,83,44,62,53,19,35,90,77,50,38,53,16,24,10,59,72,21,24,91,15,80,80,83,67,27,51,49,31,38,51,10,47,22,68,71,30,19,57,64,6,63,91,63,34,53,95,87,27,83,15,5,2,22,23,34,80,75,27,15,88,73,28,23,4,85,54,68,43,55,31,81,78,16,88,75,85,8,9,27,82,95,34,86,94,31,33,42,94,26,98,73,73,74,89,43,1,55,63,21,93,97,18,57,41,66,83,32,13,67,23,80,22,95,8,68,26,8,76,22,10,53,56,76,11,82,77,83,31,43,49,45,19,72,13,7,21,40,58,94,67,16,84,38,11,62,22,56,5,2,42,2,38,37,83,3,74,9,4,52,91,38,45,31,60,81,52,19,7,54,49,64,73,26,11,38,84,49,79,48,92,48,28,88,71,8,66,86,44,90,21,73,33,15,5,34,34,30,66,29,13,59,30,7,52,59,77,71,4,42,28,73,50,40,77,33,18,66,5,36,49,98,48,29,32,21,10,18,2,79,44,67,19,26,64,27,92,29,3,19,67,73,44,41,49,45,34,61,65,97,56,4,44,85,38,19,43,61,10,97,44,3,93,86,71,36,52,95,36,13,28,53,2,79,66,92,38,8,92,47,40,78,51,67,22,42,76,49,41,23,47,49,87,81,26,11,20,17,11,93,64,78,63,29,80,54,20,62,45,78,38,6,14,14,62,86,10,17,77,60,20,77,42,6,68,28,62,37,44,17,85,16,33,55,85,11,35,2,8,3,88,4,67,16,97,51,40,72,70,45,28,36,47,48,95,60,77,63,1,31,54,52,18,25,46,39,58,86,26,75,48,85,34,56,93,16,98,36,24,61,63,90,32,93,16,53,48,74,73,95,43,81,55,85,29,32,91,34,4,14,3,24,41,44,64,7,78,19,17,75,71,16,22,75,78,89,93,12,90,54,38,61,3,54,61,69,58,17,27,46,75,19,13,46,53,33,87,25,65,67,22,50,90,53,98,11,54,52,57,4,49,92,73,26,70,43,12,7,70,7,58,13,8,27,12,20,86,45,3,98,56,66,58,47,52,87,79,31,37,48,56,46,26,50,75,1,24,96,67,94,11,56,57,7,58,2,21,57,40,64,73,81,13,58,68,45,32,55,13,91,43,59,62,34,28,44,35,68,35,70,1,78,77,69,3,38,11,63,12,56,13,20,82,58,59,22,69,34,82,80,86,15,30,92,39,49,75,27,83,59,89,35,86,19,26,18,50,9,91,82,4,63,57,22,96,54,72,3,76,8,19,24,81,92,76,86,48,70,72,72,75,97,36,95,44,53,40,81,81,33,7,55,58,23,13,24,16,24,67,88,13,32,98,62,71,49,72,52,34,9,61,78,33,72,38,30,35,17,66,35,81,79,62,45,64,11,67,69,49,33,91,74,24,21,36,84,14,75,87,21,57,88,79,70,74,62,4,45,35,76,1,84,74,59,25,3,88,38,34,97,82,31,17,56,95,40,21,77,9,4,1,40,68,60,26,45,55,17,51,7,34,82,27,82,24,72,84,42,72,23,11,48,42,51,22,49,9,80,31,51,39,15,64,44,40,36,67,97,70,39,48,71,75,12,62,11,22,19,80,78,11,58,98,98,69,3,6,14,29,41,10,76,27,5,58,18,22,73,80,34,53,51,87,5,31,13,82,34,10,59,20,10,39,89,12,59,84,31,66,73,7,19,69,86,85,2,34,20,87,28,98,19,50,74,95,69,87,63,63,67,11,47,56,38,9,28,25,46,69,28,63,95,65,83,41,19,78,50,96,77,52,84,37,71,92,51,92,35,97,46,17,71,43,58,54,26,38,9,90,56,9,55,85,52,63,20,8,63,23,24,63,81,22,86,11,90,74,23,19,52,53,22,52,15,85,13,37,52,69,36,10,68,20,54,77,35,15,17,46,88,38,57,69,15,38,60,70,40,17,12,79,33,17,88,90,72,2,62,23,91,41,18,56,22,38,35,37,11,23,381039]
game = Game(code, screen)
game.exec()
print(screen.render())
print("part1: %i" % screen.count(2))

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                                         %
%     ## # ### # ####   ## ##  # ## ##### %
% # ###### ## ##        #     ## # #      %
% ## #   ## #   #    ### #   # ##  # # ## %
%  # #  ##### # ##   ##  ## ###  # # #  # %
% # # ###      # #  # #  #    ## ##  #    %
%   #### #### #  ## ###   ##  ##    ##  # %
%      ######  ##  ###  ### # #  # ### #  %
% #    ###  #  # #### # #    # ## ## ## # %
%  ## ## # # # #    #### # ## # ## ## ##  %
%  #  ### #  ## ##   ## ###  #    ###   # %
%  ##  # #### ######### ## ## # ## ## ### %
%   ### #    ########   #####  #    ## ## %
% #  ## #   ###### ## # ## ## #####  #    %
% #  # #  #  ##### # ## ##  ##   ## ## ## %
%                                         %
%                  *                      %
%                                         %
%                                         %
%                    =                    %
%                                         %

part1: 291


In [4]:
code2 = code.copy()
screen2 = Screen()
code2[0] = 2
game2 = Game(code2, screen2)
game2.exec1()

In [5]:
frames = 0
old_score = 0
while(True):
    frames = frames + 1
    game2.play_self(False)
    if(game2.score != old_score):
        old_score = game2.score
        if(screen2.count(2) == 0):
            break
print("frames to finish = %i" % frames)
print("part 2: score = %i" % game2.score)

halted!
frames to finish = 4080
part 2: score = 14204
